In [3]:
import re
import polars as pl
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_batch
import csv
import io

# Leitura do arquivo SQL
with open('grupodemanda_202512021151.sql', 'r', encoding='utf-8') as f:
    sql_content = f.read()

# Parser linha por linha com tratamento especial para strings
def parse_line_by_line(sql_content):
    """Parse linha por linha com tratamento especial para strings"""
    data_rows = []
    
    lines = sql_content.split('\n')
    
    for line in lines:
        line = line.strip()
        if not line.startswith('(') and '\t (' not in line and '	 (' not in line:
            if '(' not in line or not any(c.isdigit() for c in line[:20]):
                continue
        
        # Extrair tupla da linha
        if '(' in line and ')' in line:
            start = line.find('(')
            end = line.rfind(')')
            
            if start >= 0 and end > start:
                tuple_content = line[start+1:end]
                
                # Parse manual respeitando strings
                values = []
                current = ''
                in_string = False
                
                for char in tuple_content:
                    if char == "'" and not in_string:
                        in_string = True
                        current += char
                    elif char == "'" and in_string:
                        in_string = False
                        current += char
                    elif char == ',' and not in_string:
                        values.append(current.strip())
                        current = ''
                    else:
                        current += char
                
                if current:
                    values.append(current.strip())
                
                if len(values) >= 9 and values[0].isdigit():
                    data_rows.append(values)
    
    return data_rows

# Usar o parser linha por linha
data_lines = parse_line_by_line(sql_content)

print(f"Total de linhas extraídas: {len(data_lines)}")

# IDs que estavam faltando para verificar
missing_ids = [103171, 107729, 256460, 107761, 102749, 107545, 107611, 104241, 
               96208, 434600, 107728, 95463, 107738, 96209, 256427, 107734, 107779]

found_ids = [int(line[0]) for line in data_lines if line[0].isdigit()]
print(f"IDs encontrados: {len(found_ids)}")

missing_found = [id for id in missing_ids if id in found_ids]
still_missing = [id for id in missing_ids if id not in found_ids]
print(f"IDs antes faltantes agora encontrados: {missing_found}")
print(f"IDs ainda faltando: {still_missing}")

# Processar cada linha de dados
processed_data = []
for line in data_lines:
    try:
        id_val = int(line[0])
        ativo = line[1].lower() == 'true'
        data_criacao = line[2].strip("'")
        descricao = line[3].strip("'")
        nome = line[4].strip("'")
        # Usar 0 quando fundo_municipal_id for NULL (não permite null na tabela)
        fundo_municipal_id = 0 if line[7] == 'NULL' or line[7] is None or line[7] == '' else int(line[7])
        
        processed_data.append({
            'id': id_val,
            'nome': nome,
            'descricao': descricao,
            'fundo_municipal_id': fundo_municipal_id,
            'ativo': ativo,
            'data_criacao': data_criacao,
        })
    except (ValueError, IndexError) as e:
        print(f"Erro ao processar linha: {line[:3]}... - {e}")
        continue

# Criar DataFrame com Polars
df = pl.DataFrame(processed_data)

# Converter data_criacao para datetime
def parse_datetime(date_str):
    """Converte string de data para datetime, suportando com e sem milissegundos"""
    try:
        return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

df = df.with_columns(
    pl.col('data_criacao').map_elements(parse_datetime, return_dtype=pl.Datetime('us')).alias('data_criacao')
)

print(f"\nTotal de registros a importar: {len(df)}")
print("\nPrimeiros 5 registros:")
print(df.head(5))

# Conectar ao PostgreSQL e fazer o insert
try:
    conn = psycopg2.connect(
        host='localhost',
        database='agefis',
        user='postgres',
        password='postgres',
        port=5432
    )
    
    cursor = conn.cursor()
    
    # Query sem a coluna afinidades
    insert_query = """
        INSERT INTO "fiscalizacao"."grupos_ocorrencia" 
        (id, nome, descricao, fundo_municipal_id, ativo, data_criacao)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (id) DO UPDATE SET
            nome = EXCLUDED.nome,
            descricao = EXCLUDED.descricao,
            fundo_municipal_id = EXCLUDED.fundo_municipal_id,
            ativo = EXCLUDED.ativo
    """
    
    # Converter DataFrame para lista de tuplas
    data_to_insert = [
        (
            row['id'],
            row['nome'],
            row['descricao'],
            row['fundo_municipal_id'],
            row['ativo'],
            row['data_criacao']
        )
        for row in df.to_dicts()
    ]
    
    # Executar insert em batch
    execute_batch(cursor, insert_query, data_to_insert, page_size=100)
    
    conn.commit()
    print(f"\n✅ {len(data_to_insert)} registros inseridos/atualizados com sucesso!")
    
except Exception as e:
    print(f"❌ Erro ao inserir dados: {e}")
    if 'conn' in dir():
        conn.rollback()
    
finally:
    if 'cursor' in dir():
        cursor.close()
    if 'conn' in dir():
        conn.close()

Total de linhas extraídas: 172
IDs encontrados: 172
IDs antes faltantes agora encontrados: [103171, 107729, 256460, 107761, 102749, 107545, 107611, 104241, 96208, 434600, 107728, 95463, 107738, 96209, 256427, 107734, 107779]
IDs ainda faltando: []

Total de registros a importar: 172

Primeiros 5 registros:
shape: (5, 6)
┌────────┬──────────────────┬──────────────────────────┬────────────────────┬───────┬──────────────┐
│ id     ┆ nome             ┆ descricao                ┆ fundo_municipal_id ┆ ativo ┆ data_criacao │
│ ---    ┆ ---              ┆ ---                      ┆ ---                ┆ ---   ┆ ---          │
│ i64    ┆ str              ┆ str                      ┆ i64                ┆ bool  ┆ datetime[μs] │
╞════════╪══════════════════╪══════════════════════════╪════════════════════╪═══════╪══════════════╡
│ 103187 ┆ 3.10 Posturas e  ┆ Na zona de orla          ┆ 3                  ┆ true  ┆ 2017-10-20   │
│        ┆ práticas inade…  ┆                          ┆                

In [12]:
import re
import polars as pl
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_batch
import hashlib

# Leitura do arquivo SQL
with open('usuario_202512021153.sql', 'r', encoding='utf-8') as f:
    sql_content = f.read()

# Extrair blocos de INSERT
insert_blocks = re.findall(r'INSERT INTO usuario \([^)]+\) VALUES\s+(.*?)(?=INSERT INTO|$)', sql_content, re.DOTALL)

data_lines = []

for block in insert_blocks:
    # Encontrar todas as tuplas com parênteses
    pattern = r'\(([^)]+)\)'
    matches = re.findall(pattern, block)
    
    for match in matches:
        parts = [p.strip() for p in match.split(',')]
        # Verificar se é um dado válido (primeiro valor é número) e tem pelo menos as colunas esperadas
        if len(parts) >= 21 and parts[0].isdigit():  # Precisa de pelo menos até setor_id (índice 20)
            data_lines.append(parts)

print(f"Total de linhas extraídas: {len(data_lines)}")
if data_lines:
    print(f"Colunas por linha (primeiro): {len(data_lines[0])}")
    print(f"Colunas por linha (min/max): {min(len(line) for line in data_lines)}/{max(len(line) for line in data_lines)}")

# Processar dados para a tabela usuarios
usuarios_data = []
fiscais_data = []
login_counter = {}  # Para evitar logins duplicados
usuarios_ids = set()  # Rastrear IDs de usuários que serão inseridos

def parse_datetime_safe(date_str):
    """Converte string de data para datetime, suportando vários formatos"""
    if not date_str or date_str == 'NULL':
        return None
    
    date_str = date_str.strip("'")
    
    # Lista de formatos para tentar
    formats = [
        '%Y-%m-%d %H:%M:%S.%f',  # Com milissegundos
        '%Y-%m-%d %H:%M:%S',      # Sem milissegundos
        '%Y-%m-%d',                # Apenas data
    ]
    
    # Trata caso especial de data inválida com "BC"
    if 'BC' in date_str:
        return None
    
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    return None

def get_unique_login(base_login, user_id):
    """Garante que o login seja único adicionando um sufixo se necessário"""
    if base_login not in login_counter:
        login_counter[base_login] = 0
        return base_login
    
    login_counter[base_login] += 1
    # Gera um login único com sufixo
    new_login = f"{base_login}{login_counter[base_login]}"
    return new_login

for line in data_lines:
    try:
        user_id = int(line[0])
        ativo = line[1].lower() == 'true'
        data_criacao = line[2].strip("'")
        email = line[5].strip("'") if len(line) > 5 else f"user{user_id}@example.com"
        matricula = line[6].strip("'") if len(line) > 6 and line[6] != 'NULL' else None
        nome = line[8].strip("'") if len(line) > 8 else f"User {user_id}"
        senha_original = line[9].strip("'") if len(line) > 9 else "default_hash"
        lotacao = line[20].strip("'") if len(line) > 20 and line[20] != 'NULL' else None
        
        # Gerar login a partir do email (primeira parte antes do @)
        base_login = email.split('@')[0] if email else f"user_{user_id}"
        base_login = base_login.lower().replace(' ', '.').replace('_', '.')
        
        # Garantir unicidade do login
        login = get_unique_login(base_login, user_id)
        
        # Usar a senha original como password_hash (já vem hasheada do banco antigo)
        password_hash = senha_original
        
        # Adicionar à tabela usuarios
        usuarios_data.append({
            'id': user_id,
            'login': login,
            'email': email,
            'password_hash': password_hash,
            'nome': nome,
            'data_criacao': data_criacao,
            'ativo': ativo,
        })
        usuarios_ids.add(user_id)
        
        # Adicionar à tabela fiscais se matricula existir
        if matricula and matricula != '0' and matricula != '':
            fiscais_data.append({
                'id': user_id,
                'matricula': matricula,
                'lotacao': lotacao,
                'data_criacao': data_criacao,
                'ativo': ativo,
            })
        
    except (ValueError, IndexError) as e:
        continue

# Criar DataFrames com Polars
df_usuarios = pl.DataFrame(usuarios_data)
df_fiscais = pl.DataFrame(fiscais_data)

# Converter datas para datetime
def convert_dates(df, columns):
    for col in columns:
        df = df.with_columns(
            pl.col(col).map_elements(parse_datetime_safe, return_dtype=pl.Datetime('us')).alias(col)
        )
    return df

df_usuarios = convert_dates(df_usuarios, ['data_criacao'])
df_fiscais = convert_dates(df_fiscais, ['data_criacao'])

# Remover duplicatas de login (manter primeiro)
df_usuarios = df_usuarios.unique(subset=['login'], keep='first')

# Manter apenas fiscais cujos IDs estão em usuarios
valid_usuario_ids = set(df_usuarios['id'].to_list())
df_fiscais = df_fiscais.filter(pl.col('id').is_in(valid_usuario_ids))

print(f"\n📊 USUARIOS - Total de registros: {len(df_usuarios)}")
print(df_usuarios.head(3))

print(f"\n🔐 FISCAIS - Total de registros: {len(df_fiscais)}")
print(df_fiscais.head(3))

# Conectar ao PostgreSQL
try:
    conn = psycopg2.connect(
        host='localhost',
        database='agefis',
        user='postgres',
        password='postgres',
        port=5432
    )
    
    cursor = conn.cursor()
    
    # Query para inserir em usuarios
    insert_usuarios = """
        INSERT INTO "seguranca"."usuarios" 
        (id, login, email, password_hash, nome, data_criacao, ativo)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (id) DO UPDATE SET
            login = EXCLUDED.login,
            email = EXCLUDED.email,
            password_hash = EXCLUDED.password_hash,
            nome = EXCLUDED.nome,
            ativo = EXCLUDED.ativo
    """
    
    # Query para inserir em fiscais
    insert_fiscais = """
        INSERT INTO "fiscalizacao"."fiscais" 
        (id, matricula, lotacao, data_criacao, ativo)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (id) DO UPDATE SET
            matricula = EXCLUDED.matricula,
            lotacao = EXCLUDED.lotacao,
            ativo = EXCLUDED.ativo
    """
    
    # Preparar dados para usuarios
    usuarios_to_insert = [
        (
            row['id'],
            row['login'],
            row['email'],
            row['password_hash'],
            row['nome'],
            row['data_criacao'],
            row['ativo'],
        )
        for row in df_usuarios.to_dicts()
    ]
    
    # Preparar dados para fiscais
    fiscais_to_insert = [
        (
            row['id'],
            row['matricula'],
            row['lotacao'],
            row['data_criacao'],
            row['ativo'],
        )
        for row in df_fiscais.to_dicts()
    ]
    
    # Executar inserts em batch
    print("\n⏳ Inserindo em 'seguranca.usuarios'...")
    execute_batch(cursor, insert_usuarios, usuarios_to_insert, page_size=100)
    conn.commit()
    print(f"✅ {len(usuarios_to_insert)} usuários inseridos!")
    
    if fiscais_to_insert:
        print("\n⏳ Inserindo em 'fiscalizacao.fiscais'...")
        execute_batch(cursor, insert_fiscais, fiscais_to_insert, page_size=100)
        conn.commit()
        print(f"✅ {len(fiscais_to_insert)} fiscais inseridos!")
    else:
        print("\n⚠️  Nenhum fiscal para inserir")
    
except Exception as e:
    print(f"❌ Erro ao inserir dados: {e}")
    conn.rollback()
    
finally:
    cursor.close()
    conn.close()

Total de linhas extraídas: 885
Colunas por linha (primeiro): 33
Colunas por linha (min/max): 33/34

📊 USUARIOS - Total de registros: 885
shape: (3, 7)
┌─────────┬────────────────┬───────────────┬───────────────┬───────────────┬───────────────┬───────┐
│ id      ┆ login          ┆ email         ┆ password_hash ┆ nome          ┆ data_criacao  ┆ ativo │
│ ---     ┆ ---            ┆ ---           ┆ ---           ┆ ---           ┆ ---           ┆ ---   │
│ i64     ┆ str            ┆ str           ┆ str           ┆ str           ┆ datetime[μs]  ┆ bool  │
╞═════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════╡
│ 54123   ┆ silvia.maciel  ┆ silvia.maciel ┆ b06fcaa1e1d12 ┆ Silvia        ┆ 2019-01-03    ┆ true  │
│         ┆                ┆ @agefis.forta ┆ fe35424416031 ┆ Germana Luz   ┆ 10:36:23.438  ┆       │
│         ┆                ┆ leza…         ┆ 254e…         ┆ Maciel        ┆               ┆       │
│ 2165934 ┆ andre.arrais   ┆ andre.arrais